# 1) 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
## engine : encoding 문제 해결/sep : '::'
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
df_merge = pd.merge(ratings,movies,on ='movie_id',how='left')
df_merge.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [6]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 836477
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    836478 non-null  int64 
 1   movie_id   836478 non-null  int64 
 2   count      836478 non-null  int64 
 3   timestamp  836478 non-null  int64 
 4   title      836478 non-null  object
 5   genre      836478 non-null  object
dtypes: int64(4), object(2)
memory usage: 44.7+ MB


In [7]:
df_merge.describe()

,user_id,movie_id,count,timestamp
count,836478.000000,836478.000000,836478.000000,8.364780e+05
mean,3033.120626,1849.099114,3.958293,9.721628e+08
std,1729.255651,1091.870094,0.762280,1.206216e+07
min,1.000000,1.000000,3.000000,9.567039e+08
25%,1531.000000,1029.000000,3.000000,9.652795e+08
50%,3080.000000,1747.000000,4.000000,9.728388e+08
75%,4485.000000,2763.000000,5.000000,9.752064e+08
max,6040.000000,3952.000000,5.000000,1.046455e+09


## 2) 분석해 봅시다.
### - ratings에 있는 유니크한 영화 개수 
### - rating에 있는 유니크한 사용자 수 
### - 가장 인기있는 영화 30개(인기순)

In [75]:
# ratings에 있는 유니크한 영화 개수
# rating에 있는 유니크한 사용자 수
print(ratings['movie_id'].nunique())
print(ratings['user_id'].nunique())

3628
6039


In [71]:
# merge를 할 때 주의를 하자!!
# NaN이 뜰수도 있다, 어떤 방향으로 할 지도 생각해보자
df_merge.groupby(['title','movie_id'])['user_id'].count().sort_values(ascending=False).head(30)

title                                                  movie_id
American Beauty (1999)                                 2858        3211
Star Wars: Episode IV - A New Hope (1977)              260         2910
Star Wars: Episode V - The Empire Strikes Back (1980)  1196        2885
Star Wars: Episode VI - Return of the Jedi (1983)      1210        2716
Saving Private Ryan (1998)                             2028        2561
Terminator 2: Judgment Day (1991)                      589         2509
Silence of the Lambs, The (1991)                       593         2498
Raiders of the Lost Ark (1981)                         1198        2473
Back to the Future (1985)                              1270        2460
Matrix, The (1999)                                     2571        2434
Jurassic Park (1993)                                   480         2413
Sixth Sense, The (1999)                                2762        2385
Fargo (1996)                                           608         2371


In [9]:
df_merge.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [10]:
df_merge.columns

Index(['user_id', 'movie_id', 'count', 'timestamp', 'title', 'genre'], dtype='object')

In [11]:
#필요한 열만 뽑아쓴다.
df_main = df_merge[['user_id', 'movie_id','title','count']]

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.


In [12]:
my_favorite = ['Matrix, The (1999)' , 'Toy Story (1995)' ,"Bug's Life, A (1998)" ,'Gladiator (2000)' ,'Terminator 2: Judgment Day (1991)']
movie_id = [2571,1,2355,3578,589]

# 저를 6041번 user_id로 등록합니다, 지금까지 본 영화와 그 영화의 평점을 각 열에 맞게 넣어줍니다.
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': movie_id,'title': my_favorite, 'count':[5,4,3,3,3]})

#DataFrame.any() >>> boolean
if not df_main.isin({'user_id':[6041]})['user_id'].any(): 
    df_main = df_main.append(my_playlist)
    # 위에 임의로 만든 my_favorite 데이터를 추가

df_main.tail(10)       # 확인

,user_id,movie_id,title,count
836473,6040,1090,Platoon (1986),3
836474,6040,1094,"Crying Game, The (1992)",5
836475,6040,562,Welcome to the Dollhouse (1995),5
836476,6040,1096,Sophie's Choice (1982),4
836477,6040,1097,E.T. the Extra-Terrestrial (1982),4
0,6041,2571,"Matrix, The (1999)",5
1,6041,1,Toy Story (1995),4
2,6041,2355,"Bug's Life, A (1998)",3
3,6041,3578,Gladiator (2000),3
4,6041,589,Terminator 2: Judgment Day (1991),3


In [77]:
df_main.any()

user_id    True
title      True
cnt        True
dtype: bool

## 4) CSR matrix를 직접 만들어 봅시다.

In [13]:
#인덱스를 재설정 해줍니다.
df_main.reset_index(drop=True,inplace=True)
df_main.tail(10)

,user_id,movie_id,title,count
836473,6040,1090,Platoon (1986),3
836474,6040,1094,"Crying Game, The (1992)",5
836475,6040,562,Welcome to the Dollhouse (1995),5
836476,6040,1096,Sophie's Choice (1982),4
836477,6040,1097,E.T. the Extra-Terrestrial (1982),4
836478,6041,2571,"Matrix, The (1999)",5
836479,6041,1,Toy Story (1995),4
836480,6041,2355,"Bug's Life, A (1998)",3
836481,6041,3578,Gladiator (2000),3
836482,6041,589,Terminator 2: Judgment Day (1991),3


## - indexing for search

In [14]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = df_main['user_id'].unique()
movie_unique = df_main['title'].unique()

# 유저, 영화를 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
df_main['user_id'].head()

0    1
1    1
2    1
3    1
4    1
Name: user_id, dtype: int64

In [15]:
len(user_unique)

6040

In [16]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(movie_to_idx['Terminator 2: Judgment Day (1991)'])

6039
92


In [17]:
movie_to_idx.get

<function dict.get(key, default=None, /)>

In [18]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = df_main['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(df_main):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    df_main['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = df_main['title'].map(movie_to_idx.get).dropna()
if len(temp_artist_data) == len(df_main):
    print('artist column indexing OK!!')
    df_main['title'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

df_main.head()

user_id column indexing OK!!
artist column indexing OK!!


,user_id,movie_id,title,count
0,0,1193,0,5
1,0,661,1,3
2,0,914,2,3
3,0,3408,3,4
4,0,2355,4,5


In [19]:
df_main = df_main[['user_id','title','count']]

In [20]:
df_main.rename(columns={'count':'cnt'}, inplace=True)

## 4) CSR matrix를 직접 만들어 봅시다.

In [21]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = df_main['user_id'].nunique()
num_movie = df_main['title'].nunique()

csr_data = csr_matrix((df_main['cnt'], (df_main.user_id, df_main.title)), shape= (num_user, num_movie))
print(csr_data)

(0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 4)	3
  (6039, 40)	4
  (6039, 67)	3
  (6039, 92)	3
  (6039, 124)	5


## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [55]:
# Implicit AlternatingLeastSquares 모델의 선언
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 3. use_gpu : GPU를 사용할 것인지 4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [56]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [57]:
# 모델 훈련
als_model.fit(csr_data_transpose)

100%|██████████| 15/15 [00:01<00:00, 14.74it/s]


## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [58]:
gm, mtx = user_to_idx[6041], movie_to_idx['Matrix, The (1999)']
gm_vector = als_model.user_factors[gm]
mtx_vector = als_model.item_factors[mtx]

In [60]:
gm_vector.shape

(100,)

In [61]:
mtx_vector.shape

(100,)

In [62]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(gm_vector, mtx_vector)

0.5373619

In [63]:
six = movie_to_idx['Sixth Sense, The (1999)']
six_vector = als_model.item_factors[six]
np.dot(gm_vector, six_vector)

0.23363763

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.


In [64]:
favorite_movie = 'Matrix, The (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(124, 0.17091532),
 (92, 0.13347109),
 (62, 0.11994326),
 (141, 0.0959421),
 (200, 0.09509552),
 (145, 0.093379386),
 (107, 0.088648684),
 (375, 0.08692173),
 (175, 0.08460467),
 (317, 0.08180158),
 (75, 0.0761722),
 (44, 0.06923527),
 (431, 0.06618537),
 (150, 0.06384395),
 (193, 0.063732974)]

In [65]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Fifth Element, The (1997)',
 'Jurassic Park (1993)',
 'Face/Off (1997)',
 'Men in Black (1997)',
 'Twelve Monkeys (1995)',
 'Hunt for Red October, The (1990)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Gattaca (1997)',
 'Independence Day (ID4) (1996)',
 'Alien (1979)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [67]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.52622014),
 (372, 0.3355266),
 (107, 0.3094297),
 (62, 0.30925116),
 (87, 0.3037362),
 (141, 0.29330897),
 (200, 0.27942327),
 (175, 0.26433712),
 (346, 0.23928761),
 (126, 0.23891559),
 (48, 0.23863822),
 (33, 0.23861736),
 (38, 0.23363765),
 (110, 0.2323528),
 (322, 0.22448196),
 (75, 0.22149742),
 (474, 0.21108136),
 (482, 0.1919908),
 (193, 0.19079456),
 (22, 0.18736658)]

In [68]:
[idx_to_movie[i[0]] for i in artist_recommended]

['Toy Story 2 (1999)',
 'X-Men (2000)',
 'Jurassic Park (1993)',
 'Total Recall (1990)',
 'Braveheart (1995)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Men in Black (1997)',
 'Patriot, The (2000)',
 'Shakespeare in Love (1998)',
 'Saving Private Ryan (1998)',
 'Aladdin (1992)',
 'Sixth Sense, The (1999)',
 'Groundhog Day (1993)',
 'Babe (1995)',
 'Hunt for Red October, The (1990)',
 'Chicken Run (2000)',
 'Mission: Impossible 2 (2000)',
 'Alien (1979)',
 'Back to the Future (1985)']

In [69]:
Xman = movie_to_idx['X-Men (2000)']
explain = als_model.explain(user, csr_data, itemid=Xman)

In [70]:
# 추천에 기여한 정도(합이 콘텐츠의 점수)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Gladiator (2000)', 0.16828316487894465),
 ('Matrix, The (1999)', 0.11284330440233092),
 ("Bug's Life, A (1998)", 0.023402918532595195),
 ('Terminator 2: Judgment Day (1991)', 0.015875395770502926),
 ('Toy Story (1995)', 0.006838643852985306)]

In [ ]:
### 평점 혹은 view를 이용한 데이터와 다른 데이터(장르)도 고려하는 것이 좋다 생각했다.
### ALS모델에 대해 간략히 나왔는데 이를 정리해보고 싶다.  